# Prerequisites

In [1]:
import json
import os

import CNN
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F
from data import get_dataloaders
from EarlyStopping import EarlyStopper
from pick import pick
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from tqdm import tqdm

sns.set_style("whitegrid")

# Load stats

In [6]:
batch_size=64
num_workers = 1
data_augmentation = False

train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
    batch_size, num_workers=num_workers, data_augmentation=data_augmentation)

In [10]:
models_dir = '../models'
json_files = []

# Find jsons
for model_name in os.listdir(models_dir):
    model_dir = os.path.join(models_dir, model_name)
    if os.path.isdir(model_dir):
        stats_dir = os.path.join(model_dir, 'stats')
        for file in os.listdir(stats_dir):
            if file.endswith('.json'):
                json_files.append(os.path.join(stats_dir, file))

# Load stats
stats_list = []
for fname in json_files:
    with open(fname, 'r') as f:
        stats_list.append(json.load(f))

# json_files

# Model metrics

In [9]:
# Check if cuda is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using device:\t{device}')

in_size = (64, 64)  # h, w
def get_model(model_type):
    if model_type.lower() == 'resnet':
        model = CNN.ResNet(3,
                           32,
                           in_size,
                           num_res_blocks=9,
                           dropout=dropout,
                           BN=BN)
    elif model_type.lower() == 'cnn_4':
        model = CNN.CNN_4(3, in_size, dropout=dropout, BN=BN)
    elif model_type.lower() == 'rn18_freeze':
        model = CNN.RN18(True)
    elif model_type.lower() == 'rn18':
        model = CNN.RN18(False)
    model.to(device)

Using device:	cpu


In [16]:
for stats in stats_list:
    model_type = stats['model']
    model_name = stats['model_name']
    print(model_name)
    for k in ['data_augmentation', 'optimizer', 'batch_norm']:
        print(f'{k[:10]}:\t{stats[k]}')
    print('\n')

    checkpoint = f'{models_dir}/{model_type}/checkpoints/{model_name}.pt'
    

cnn_4_2
data_augme:	True
optimizer:	ADAM
batch_norm:	False


../models/cnn_4/checkpoints/cnn_4_2.pt
cnn_4_3
data_augme:	True
optimizer:	SGD
batch_norm:	True


../models/cnn_4/checkpoints/cnn_4_3.pt
cnn_4_0
data_augme:	True
optimizer:	ADAM
batch_norm:	True


../models/cnn_4/checkpoints/cnn_4_0.pt
cnn_4_1
data_augme:	False
optimizer:	ADAM
batch_norm:	True


../models/cnn_4/checkpoints/cnn_4_1.pt
resnet9_0
data_augme:	True
optimizer:	ADAM
batch_norm:	True


../models/resnet/checkpoints/resnet9_0.pt
resnet9_2
data_augme:	True
optimizer:	ADAM
batch_norm:	False


../models/resnet/checkpoints/resnet9_2.pt
resnet9_1
data_augme:	False
optimizer:	ADAM
batch_norm:	True


../models/resnet/checkpoints/resnet9_1.pt
resnet9_3
data_augme:	True
optimizer:	SGD
batch_norm:	True


../models/resnet/checkpoints/resnet9_3.pt
rn18_freeze_0
data_augme:	True
optimizer:	ADAM
batch_norm:	True


../models/rn18_freeze/checkpoints/rn18_freeze_0.pt
rn18_0
data_augme:	True
optimizer:	ADAM
batch_norm:	True


../models/

In [ ]:
all_labels = [label for _, label in test_dataset]
sum(all_labels) / len(all_labels)